In [148]:
# Import all libraries here
import random
random.seed(88)
import pandas as pd
import numpy as np
np.random.seed(88)
import tensorflow as tf
tf.random.set_seed(88)
from tensorflow import keras
keras.utils.set_random_seed(88)
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib import pyplot as plt
from keras import backend as K

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read Data

In [149]:
# Output format
# Description: The file contains days, hours, and base stations for which we
# need energy consumption estimation and will be used for scoring predictions.
power_pred = pd.read_csv('/Users/davidliu/Desktop/AI:5G Challenge/Submission.csv')
power_pred

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1
2,2023-01-01 12:00:00,B_0,NaN,1
3,2023-01-01 13:00:00,B_0,NaN,1
4,2023-01-01 23:00:00,B_0,NaN,1
...,...,...,...,...
26134,2023-01-02 19:00:00,B_1019,NaN,5
26135,2023-01-02 20:00:00,B_1019,NaN,5
26136,2023-01-02 21:00:00,B_1019,NaN,5
26137,2023-01-02 22:00:00,B_1019,NaN,5


In [151]:
# Dataset 1
# Description: Energy consumption data (ECdata.csv): include hour-level energy
# consumption specifications (e.g., total energy consumption of the base stations).
ECdata = pd.read_csv('/Users/davidliu/Desktop/AI:5G Challenge/data1.csv')
ECdata

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.275037
1,1/1/2023 2:00,B_0,55.904335
2,1/1/2023 3:00,B_0,57.698057
3,1/1/2023 4:00,B_0,55.156951
4,1/1/2023 5:00,B_0,56.053812
...,...,...,...
92624,1/2/2023 17:00,B_1018,14.648729
92625,1/2/2023 18:00,B_1018,14.648729
92626,1/2/2023 21:00,B_1018,13.452915
92627,1/2/2023 22:00,B_1018,13.602392


In [152]:
# Dataset 2
# Description: Cell-level data (CLdata.csv): include hour-level counters,
# including service compliance counters (e.g., load) and energy-saving methods
# counters (e.g., duration of energy saving mode activation).
CLdata = pd.read_csv('/Users/davidliu/Desktop/AI:5G Challenge/data2.csv')
CLdata

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
125570,1/2/2023 19:00,B_745,Cell3,0.071000,0.0,0.0,0.0,0,0.0,0.0
125571,1/2/2023 20:00,B_745,Cell3,0.064962,0.0,0.0,0.0,0,0.0,0.0
125572,1/2/2023 21:00,B_745,Cell3,0.081865,0.0,0.0,0.0,0,0.0,0.0
125573,1/2/2023 22:00,B_745,Cell3,0.097615,0.0,0.0,0.0,0,0.0,0.0


In [153]:
# Dataset 3
# Description: Base Station basic information (BSinfo.csv): include
# configuration parameters and hardware attributes
BSinfo = pd.read_csv('/Users/davidliu/Desktop/AI:5G Challenge/data3.csv')
BSinfo

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.000,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.000,20,4,6.875934
2,B_2,Cell0,Type1,Mode2,365.000,20,4,6.875934
3,B_3,Cell0,Type2,Mode2,532.000,20,4,6.875934
4,B_4,Cell0,Type2,Mode2,532.000,20,4,6.875934
...,...,...,...,...,...,...,...,...
1212,B_925,Cell1,Type8,Mode1,697.002,10,4,7.877728
1213,B_105,Cell2,Type1,Mode2,426.980,2,2,6.877429
1214,B_745,Cell2,Type1,Mode2,426.980,2,2,6.877429
1215,B_105,Cell3,Type1,Mode2,426.980,2,2,6.877429


# Clean and Combine Data

In [154]:
print(len(BSinfo['BS'].unique())) #data3
print(len(CLdata['BS'].unique())) #data2
print(len(ECdata['BS'].unique())) #data1

1020
1020
923


### Merge data sets 1+2+3

In [155]:
merged_data = ECdata.merge(CLdata, on=["Time", "BS"], how="inner")
final_merged_data = merged_data.merge(BSinfo, on="BS", how="inner")

In [156]:
data = final_merged_data.copy()

In [157]:
# Convert date strings to datetime objects
data['Time'] = pd.to_datetime(data['Time'])

# Sort the dataset by date
data.sort_values(by='Time', inplace=True)

# Split Year, Month, Day, and Hour
data['Year'] = data['Time'].dt.year
data['Month'] = data['Time'].dt.month
data['Day'] = data['Time'].dt.day
data['Hour'] = data['Time'].dt.hour

In [158]:
data

,Time,BS,Energy,CellName_x,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,...,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower,Year,Month,Day,Hour
0,2023-01-01 01:00:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.0,0,0.0,...,Type1,Mode2,365.0,20,4,6.875934,2023,1,1,1
56593,2023-01-01 01:00:00,B_384,25.112108,Cell0,0.404980,0.0,0.0,0.0,0,0.0,...,Type6,Mode2,189.0,10,2,6.427504,2023,1,1,1
97475,2023-01-01 01:00:00,B_659,9.267564,Cell0,0.024830,0.0,0.0,0.0,0,0.0,...,Type4,Mode2,532.0,20,2,6.875934,2023,1,1,1
97596,2023-01-01 01:00:00,B_660,8.819133,Cell0,0.011900,0.0,0.0,0.0,0,0.0,...,Type4,Mode2,532.0,20,2,6.875934,2023,1,1,1
97709,2023-01-01 01:00:00,B_661,17.937220,Cell0,0.080040,0.0,0.0,0.0,0,0.0,...,Type6,Mode2,189.0,10,2,6.427504,2023,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86764,2023-01-08 00:00:00,B_584,29.297459,Cell0,0.121440,0.0,0.0,0.0,0,0.0,...,Type3,Mode2,155.6,10,2,6.427504,2023,1,8,0
86765,2023-01-08 00:00:00,B_584,29.297459,Cell0,0.121440,0.0,0.0,0.0,0,0.0,...,Type3,Mode2,189.0,10,2,6.427504,2023,1,8,0
117363,2023-01-08 00:00:00,B_783,45.590433,Cell0,0.970287,0.0,0.0,0.0,0,0.0,...,Type7,Mode2,365.0,20,2,6.875934,2023,1,8,0
117473,2023-01-08 00:00:00,B_784,31.390135,Cell0,0.690920,0.0,0.0,0.0,0,0.0,...,Type4,Mode2,532.0,20,2,6.875934,2023,1,8,0


In [159]:
print(len(final_merged_data['BS'].unique()))
print(len(final_merged_data['Time'].unique()))
print(len(final_merged_data['Energy'].unique()))
print(len(final_merged_data['CellName_x'].unique()))
print(len(final_merged_data['CellName_x'].unique()))

923
168
612
4
4


# Train Model

In [160]:
print(data.shape) #data dimension
print(list(data)) #column names

(125690, 22)
['Time', 'BS', 'Energy', 'CellName_x', 'load', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4', 'ESMode5', 'ESMode6', 'CellName_y', 'RUType', 'Mode', 'Frequency', 'Bandwidth', 'Antennas', 'TXpower', 'Year', 'Month', 'Day', 'Hour']


In [161]:
# Extract target variable 'Energy'
y = data['Energy']

# Drop unnecessary columns (e.g., 'Time', 'Energy')
data.drop(columns=['Time', 'Energy'], inplace=True)

# One-hot encode categorical features
categorical_cols = ['BS', 'CellName_x', 'CellName_y', 'RUType', 'Mode', ]
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
data_encoded[data_encoded.columns] = scaler.fit_transform(data_encoded[data_encoded.columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data_encoded, y, test_size=0.2, shuffle=False)

# Convert data to numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
# Time series transformation
X_train_ts = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_ts = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Tensor LSTM - Time Series

In [162]:
# LSTM Model - Time series
lstm_model = keras.Sequential([
    layers.LSTM(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
lstm_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = lstm_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = lstm_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

lstm_model.summary()

Epoch 1/50
2514/2514 [==============================] - 16s 6ms/step - loss: 26.8239 - mean_absolute_percentage_error: 26.8239 - val_loss: 8.9957 - val_mean_absolute_percentage_error: 8.9957
Epoch 2/50
2514/2514 [==============================] - 14s 5ms/step - loss: 14.4282 - mean_absolute_percentage_error: 14.4282 - val_loss: 7.7412 - val_mean_absolute_percentage_error: 7.7412
Epoch 3/50
2514/2514 [==============================] - 12s 5ms/step - loss: 12.7652 - mean_absolute_percentage_error: 12.7652 - val_loss: 7.2866 - val_mean_absolute_percentage_error: 7.2866
Epoch 4/50
2514/2514 [==============================] - 14s 6ms/step - loss: 11.5683 - mean_absolute_percentage_error: 11.5683 - val_loss: 7.1268 - val_mean_absolute_percentage_error: 7.1268
Epoch 5/50
2514/2514 [==============================] - 10s 4ms/step - loss: 10.6069 - mean_absolute_percentage_error: 10.6069 - val_loss: 6.7534 - val_mean_absolute_percentage_error: 6.7534
Epoch 6/50
2514/2514 [=======================

# Pytorch LSTM - Time Series

In this PyTorch code:

We define a custom LSTMModel class that replicates the structure of your Keras model.

We instantiate the model with the specified input size and hidden size.

We define the loss function (L1 loss, equivalent to mean absolute error) and the optimizer (Adam).

We convert your training and testing data to PyTorch tensors.

We implement the training loop, including forward and backward passes.

We evaluate the model on the test set and calculate the MAPE.

We print the model summary to display its architecture.



In [163]:
# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Reshape the input data for the LSTM model
X_train_lstm = X_train.view(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.view(X_test.shape[0], 1, X_test.shape[1])

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])  # Get the output of the last time step
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# Initialize the LSTM model
input_size = X_train_lstm.shape[2]
hidden_size = 128
num_layers = 1
lstm_model = LSTMModel(input_size, hidden_size, num_layers)

# Define the loss function and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error (MAE) loss
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# Training parameters
num_epochs = 50 # training loop
batch_size = 256

# Create DataLoader for training
train_dataset = TensorDataset(X_train_lstm, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = lstm_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
lstm_model.eval()
with torch.no_grad():
    y_pred = lstm_model(X_test_lstm)

# Calculate MAPE
y_test = y_test.numpy()
y_pred = y_pred.numpy()
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {mape:.5f}%')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch [1/50], Loss: 13.5768
Epoch [2/50], Loss: 12.8722
Epoch [3/50], Loss: 14.1907
Epoch [4/50], Loss: 12.8581
Epoch [5/50], Loss: 13.5099
Epoch [6/50], Loss: 13.7768
Epoch [7/50], Loss: 12.3190
Epoch [8/50], Loss: 12.9596
Epoch [9/50], Loss: 12.6848
Epoch [10/50], Loss: 13.1365
Epoch [11/50], Loss: 12.5753
Epoch [12/50], Loss: 13.1725
Epoch [13/50], Loss: 12.4786
Epoch [14/50], Loss: 14.1438
Epoch [15/50], Loss: 13.6505
Epoch [16/50], Loss: 13.6088
Epoch [17/50], Loss: 13.7501
Epoch [18/50], Loss: 13.4734
Epoch [19/50], Loss: 12.2085
Epoch [20/50], Loss: 13.0761
Epoch [21/50], Loss: 12.5626
Epoch [22/50], Loss: 15.4011
Epoch [23/50], Loss: 13.3934
Epoch [24/50], Loss: 13.5899
Epoch [25/50], Loss: 13.0920
Epoch [26/50], Loss: 13.6106
Epoch [27/50], Loss: 12.7487
Epoch [28/50], Loss: 12.6990
Epoch [29/50], Loss: 13.6820
Epoch [30/50], Loss: 12.8411
Epoch [31/50], Loss: 14.4963
Epoch [32/50], Loss: 13.2147
Epoch [33/50], Loss: 12.8594
Epoch [34/50], Loss: 13.1421
Epoch [35/50], Loss: 13

In [164]:
# Convert data to PyTorch tensors
X = torch.tensor(X_train_ts, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test_ts, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = torch.relu(out[:, -1, :])  # Select the last time step's output
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = self.fc4(out)
        return out

input_size = X.shape[2]
hidden_size = 128
num_layers = 1  # You can increase this if needed
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X), batch_size):
        inputs = X[i:i+batch_size]
        labels = y[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    mape = mean_absolute_percentage_error(y_test.numpy(), y_pred.numpy())
    print(f'MAPE on test set: {100 * mape:.5f}')

# Print model summary
print(model)


/var/folders/8x/mvv8r7hx19d1h00xq1yg_d5r0000gn/T/ipykernel_94572/3675296800.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y_train, dtype=torch.float32)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please

Epoch [1/50], Loss: 15.4164, Val Loss: 12.5521
Epoch [2/50], Loss: 15.2801, Val Loss: 12.5165
Epoch [3/50], Loss: 15.2720, Val Loss: 12.5082
Epoch [4/50], Loss: 15.3151, Val Loss: 12.5086
Epoch [5/50], Loss: 15.2127, Val Loss: 12.5075
Epoch [6/50], Loss: 15.2171, Val Loss: 12.5095
Epoch [7/50], Loss: 15.2181, Val Loss: 12.5117
Epoch [8/50], Loss: 15.2366, Val Loss: 12.5147
Epoch [9/50], Loss: 15.1608, Val Loss: 12.5157
Epoch [10/50], Loss: 15.1091, Val Loss: 12.5179
Epoch [11/50], Loss: 15.1364, Val Loss: 12.5208
Epoch [12/50], Loss: 15.1584, Val Loss: 12.5235
Epoch [13/50], Loss: 15.1598, Val Loss: 12.5232
Epoch [14/50], Loss: 15.1511, Val Loss: 12.5277
Epoch [15/50], Loss: 15.1588, Val Loss: 12.5293
Epoch [16/50], Loss: 15.1471, Val Loss: 12.5297
Epoch [17/50], Loss: 15.1014, Val Loss: 12.5324
Epoch [18/50], Loss: 15.0460, Val Loss: 12.5361
Epoch [19/50], Loss: 15.0112, Val Loss: 12.5393
Epoch [20/50], Loss: 15.0469, Val Loss: 12.5439
Epoch [21/50], Loss: 15.0590, Val Loss: 12.5467
E

# GRU Model - Time series Tensor

In [ ]:
# GRU Model - Time series
gru_model = keras.Sequential([
    layers.GRU(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
gru_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = gru_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = gru_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

gru_model.summary()

# GRU Model - Time Series Pytorch

In [168]:
# GRU Model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = torch.relu(out[:, -1, :])  # Select the last time step's output
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = self.fc4(out)
        return out

input_size = X.shape[2]
hidden_size = 128
num_layers = 1  # You can increase this if needed
output_size = 1

model = GRUModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X), batch_size):
        inputs = X[i:i+batch_size]
        labels = y[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    mape = mean_absolute_percentage_error(y_test.numpy(), y_pred.numpy())
    print(f'MAPE on test set: {100 * mape:.5f}')

# Print model summary
print(model)


Epoch [1/50], Loss: 15.4178, Val Loss: 12.5738
Epoch [2/50], Loss: 15.2972, Val Loss: 12.5429
Epoch [3/50], Loss: 15.2159, Val Loss: 12.5270
Epoch [4/50], Loss: 15.2737, Val Loss: 12.5248
Epoch [5/50], Loss: 15.2975, Val Loss: 12.5214
Epoch [6/50], Loss: 15.2498, Val Loss: 12.5210
Epoch [7/50], Loss: 15.2881, Val Loss: 12.5242
Epoch [8/50], Loss: 15.2936, Val Loss: 12.5254
Epoch [9/50], Loss: 15.1838, Val Loss: 12.5280
Epoch [10/50], Loss: 15.2070, Val Loss: 12.5312
Epoch [11/50], Loss: 15.1530, Val Loss: 12.5328
Epoch [12/50], Loss: 15.1607, Val Loss: 12.5362
Epoch [13/50], Loss: 15.1466, Val Loss: 12.5376
Epoch [14/50], Loss: 15.1018, Val Loss: 12.5360
Epoch [15/50], Loss: 15.1050, Val Loss: 12.5376
Epoch [16/50], Loss: 15.0612, Val Loss: 12.5402
Epoch [17/50], Loss: 15.0330, Val Loss: 12.5419
Epoch [18/50], Loss: 15.0604, Val Loss: 12.5431
Epoch [19/50], Loss: 15.0518, Val Loss: 12.5475
Epoch [20/50], Loss: 15.0583, Val Loss: 12.5503
Epoch [21/50], Loss: 15.0384, Val Loss: 12.5527
E

# RNN Model - Time series Tensor

In [ ]:
# RNN Model - Time series
rnn_model = keras.Sequential([
    layers.GRU(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
rnn_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = rnn_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = rnn_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

rnn_model.summary()

# RNN Model - Time series Pytorch

In [169]:
# RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = torch.relu(out[:, -1, :])  # Select the last time step's output
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = self.fc4(out)
        return out

input_size = X.shape[2]
hidden_size = 128
num_layers = 1  # You can increase this if needed
output_size = 1

model = RNNModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X), batch_size):
        inputs = X[i:i+batch_size]
        labels = y[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    mape = mean_absolute_percentage_error(y_test.numpy(), y_pred.numpy())
    print(f'MAPE on test set: {100 * mape:.5f}')

# Print model summary
print(model)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broa

Epoch [1/50], Loss: 15.0670, Val Loss: 12.5249
Epoch [2/50], Loss: 15.0250, Val Loss: 12.5134
Epoch [3/50], Loss: 14.9837, Val Loss: 12.5138
Epoch [4/50], Loss: 14.9836, Val Loss: 12.5097
Epoch [5/50], Loss: 15.0749, Val Loss: 12.5125
Epoch [6/50], Loss: 14.9879, Val Loss: 12.5142
Epoch [7/50], Loss: 15.0360, Val Loss: 12.5146
Epoch [8/50], Loss: 15.1593, Val Loss: 12.5148
Epoch [9/50], Loss: 15.0690, Val Loss: 12.5165
Epoch [10/50], Loss: 15.0666, Val Loss: 12.5144
Epoch [11/50], Loss: 15.1179, Val Loss: 12.5157
Epoch [12/50], Loss: 15.1289, Val Loss: 12.5193
Epoch [13/50], Loss: 15.0781, Val Loss: 12.5191
Epoch [14/50], Loss: 15.1400, Val Loss: 12.5203
Epoch [15/50], Loss: 15.0996, Val Loss: 12.5206
Epoch [16/50], Loss: 15.1419, Val Loss: 12.5180
Epoch [17/50], Loss: 15.1281, Val Loss: 12.5178
Epoch [18/50], Loss: 15.0873, Val Loss: 12.5208
Epoch [19/50], Loss: 15.0928, Val Loss: 12.5201
Epoch [20/50], Loss: 15.0798, Val Loss: 12.5207
Epoch [21/50], Loss: 15.1269, Val Loss: 12.5210
E

# Regression Model - Regression Tensor

In [ ]:
# Regression Model - Regression
model = keras.Sequential([
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

model.summary()

# Regression Model - Regression Pytorch

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

# Assuming you have your data loaded and preprocessed as X and y

# Convert data to PyTorch tensors
X = torch.tensor(X_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Regression Model
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(RegressionModel, self).__init__()
        layers = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_sizes[i]))
            layers.append(nn.Dropout(0.2))
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        out = x  # Initialize the output as the input
        for layer in self.layers:
            if isinstance(layer, nn.BatchNorm1d):
                out = out.view(out.size(0), -1)  # Flatten the output if it's a BatchNorm layer
            out = layer(out)
        return out


input_size = X.shape[1]
#hidden_sizes = 128 # You can adjust the number of hidden layers and units as needed
output_size = 1
# Define the number of hidden units in a single hidden layer
hidden_units = 128  # Adjust as needed
# Create a list with a single hidden layer
hidden_sizes = [hidden_units]
model = RegressionModel(input_size, hidden_sizes, output_size)

# Loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X), batch_size):
        inputs = X[i:i+batch_size]
        labels = y[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    mape = mean_absolute_percentage_error(y_test.numpy(), y_pred.numpy())
    print(f'MAPE on test set: {100 * mape:.5f}')

# Print model summary
print(model)


/var/folders/8x/mvv8r7hx19d1h00xq1yg_d5r0000gn/T/ipykernel_94572/2463603898.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_train, dtype=torch.float32)
/var/folders/8x/mvv8r7hx19d1h00xq1yg_d5r0000gn/T/ipykernel_94572/2463603898.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y_train, dtype=torch.float32)
/var/folders/8x/mvv8r7hx19d1h00xq1yg_d5r0000gn/T/ipykernel_94572/2463603898.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
/var/folders

Epoch [1/50], Loss: 16.8090, Val Loss: 12.6871
Epoch [2/50], Loss: 15.8130, Val Loss: 12.6389
Epoch [3/50], Loss: 16.5778, Val Loss: 12.6193
Epoch [4/50], Loss: 16.3737, Val Loss: 12.6235
Epoch [5/50], Loss: 16.4898, Val Loss: 12.6231
Epoch [6/50], Loss: 16.3821, Val Loss: 12.6152
Epoch [7/50], Loss: 15.9069, Val Loss: 12.6262
Epoch [8/50], Loss: 16.5082, Val Loss: 12.6243
Epoch [9/50], Loss: 16.4238, Val Loss: 12.6179
Epoch [10/50], Loss: 16.3801, Val Loss: 12.6249
Epoch [11/50], Loss: 16.0011, Val Loss: 12.6179
Epoch [12/50], Loss: 16.1292, Val Loss: 12.6284
Epoch [13/50], Loss: 16.4738, Val Loss: 12.6178
Epoch [14/50], Loss: 16.7223, Val Loss: 12.6280
Epoch [15/50], Loss: 16.6337, Val Loss: 12.6204
Epoch [16/50], Loss: 16.7103, Val Loss: 12.6190
Epoch [17/50], Loss: 16.8976, Val Loss: 12.6136
Epoch [18/50], Loss: 16.4414, Val Loss: 12.6193
Epoch [19/50], Loss: 16.4943, Val Loss: 12.6204
Epoch [20/50], Loss: 16.5340, Val Loss: 12.6156
Epoch [21/50], Loss: 16.3369, Val Loss: 12.6178
E